In [1]:
import numpy as np

In [2]:
X_train = np.array([
    [0, 1, 1],
    [0, 0, 1],
    [0, 0, 0],
    [1, 1, 0]
])
Y_train = ['Y', 'N', 'Y', 'Y']
X_test = np.array([[1, 1, 0]])

In [3]:
def get_label_indices(labels):
    from collections import defaultdict
    label_indices = defaultdict(list)
    for index, label in enumerate(labels):
        label_indices[label].append(index)
    return label_indices

In [4]:
label_indices = get_label_indices(Y_train)
label_indices

defaultdict(list, {'Y': [0, 2, 3], 'N': [1]})

In [5]:
def get_prior(label_indices):
    prior = {label: len(indices) for label, indices in label_indices.items()}
    total_count = sum(prior.values())
    for label in prior:
        prior[label] /= total_count
    return prior

In [6]:
prior = get_prior(label_indices)
prior

{'Y': 0.75, 'N': 0.25}

In [7]:
def get_likelihood(features, label_indices, smoothing=0):
    likelihood = {}
    for label, indices in label_indices.items():
        likelihood[label] = features[indices, :].sum(axis=0) + smoothing
        total_count = len(indices)
        likelihood[label] = likelihood[label] / (total_count + 2 * smoothing)
        
    return likelihood

In [8]:
likelihood = get_likelihood(X_train, label_indices, smoothing=1)
likelihood

{'Y': array([0.4, 0.6, 0.4]), 'N': array([0.33333333, 0.33333333, 0.66666667])}

In [10]:
def get_posterior(X, prior, likelihood):
    posteriors = []
    for x in X:
        posterior = prior.copy()
        for label, likelihood_label in likelihood.items():
            for index, bool_value in enumerate(x):
                posterior[label] *= likelihood_label[index] if bool_value else (1 - likelihood_label[index])
        # normalization
        sum_posterior = sum(posterior.values())
        for label in posterior:
            if posterior[label] == float('inf'):
                posterior[label] = 1.0
            else:
                posterior[label] /= sum_posterior
        posteriors.append(posterior.copy())
    return posteriors

In [11]:
posterior = get_posterior(X_test, prior, likelihood)
posterior

[{'Y': 0.9210360075805433, 'N': 0.07896399241945673}]

### Realization of the same algorithm usin `scikit-learn` package

In [12]:
from sklearn.naive_bayes import BernoulliNB

In [15]:
# create instance
clf = BernoulliNB(alpha=1.0, fit_prior=True)

In [16]:
clf.fit(X_train, Y_train)

BernoulliNB()

In [17]:
pred_prob = clf.predict_proba(X_test)
pred_prob

array([[0.07896399, 0.92103601]])

In [19]:
pred = clf.predict(X_test)
print(pred)

['Y']
